# Pulse Watson ML 

## Import, Clean, and Analyze Data
We are going to use PixieDust to load and visualize our data.

In [1]:
import pixiedust

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [2]:
df = pixiedust.sampleData("https://raw.githubusercontent.com/hovig/gyro-watson-ml/master/pulse-rate.csv")

Downloaded 45 bytes
Creating pySpark DataFrame for 'https://raw.githubusercontent.com/hovig/gyro-watson-ml/master/pulse-rate.csv'. Please wait...
Loading file using 'SparkSession'
Successfully created pySpark DataFrame for 'https://raw.githubusercontent.com/hovig/gyro-watson-ml/master/pulse-rate.csv'


In [3]:
display(df)

Heartbeats,Seconds,Bpm
7,6,70
6,6,60
5,6,60


## Build a Machine Learning Model with Spark ML 

In [4]:
df.show()

+----------+-------+---+
|Heartbeats|Seconds|Bpm|
+----------+-------+---+
|         7|      6| 70|
|         6|      6| 60|
|         5|      6| 60|
+----------+-------+---+



In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [6]:
assembler = VectorAssembler(inputCols=['Heartbeats','Seconds'],outputCol="features")
lr = LinearRegression(labelCol='Bpm', featuresCol='features')
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(df)

### Test the Model

In [7]:
def get_prediction(heart_beats, time_in_secs):
    request_df = spark.createDataFrame([(heart_beats, time_in_secs)], ['Heartbeats','Seconds'])
    response_df = model.transform(request_df)
    return response_df

In [8]:
response = get_prediction(6, 6)
response.show()

+----------+-------+---------+-----------------+
|Heartbeats|Seconds| features|       prediction|
+----------+-------+---------+-----------------+
|         6|      6|[6.0,6.0]|63.33333333333334|
+----------+-------+---------+-----------------+



# Deploy Model to Watson ML

In [9]:
import json
import requests
import urllib3

### Fill in Your Watson ML Credentials

In [10]:
service_path = 'https://ibm-watson-ml.mybluemix.net'
username = ''
password = ''
instance_id = ''
model_name = 'Pulse Rates Model'
deployment_name = 'Pulse Rates Deployment'

In [11]:
# The code was removed by DSX for sharing.

### Initialize Watson ML Repository Libraries

In [12]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

### Save Model to Watson ML Repository

In [13]:
pipeline_artifact = MLRepositoryArtifact(pipeline, name="pipeline")
model_artifact = MLRepositoryArtifact(model, training_data=df, name=model_name, pipeline_artifact=pipeline_artifact)
saved_model = ml_repository_client.models.save(model_artifact)
model_id = saved_model.uid

In [14]:
headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
ml_token = 'Bearer ' + json.loads(response.text).get('token')

### Create a Deployment

In [15]:
deployment_url = service_path + "/v3/wml_instances/" + instance_id + "/published_models/" + model_id + "/deployments/"
deployment_header = {'Content-Type': 'application/json', 'Authorization': ml_token}
deployment_payload = {"type": "online", "name": deployment_name}
deployment_response = requests.post(deployment_url, json=deployment_payload, headers=deployment_header)
deployment_response.text
scoring_url = json.loads(deployment_response.text).get('entity').get('scoring_url')
scoring_url

'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6557d1-3406-429e-bc61-6f8a1e78ffd8/published_models/42c5131f-2957-4b1b-b993-08b95d0e7827/deployments/e87c30a5-23b3-4c92-b546-1ecffd559f0d/online'

### Test the Deployment

In [16]:
def get_prediction_from_watson_ml(heart_beats, time_in_secs):
    scoring_header = {'Content-Type': 'application/json', 'Authorization': ml_token}
    scoring_payload = {'fields': ['Heartbeats','Seconds'], 'values': [[heart_beats, time_in_secs]]}
    scoring_response = requests.post(scoring_url, json=scoring_payload, headers=scoring_header)
    return scoring_response.text

In [18]:
response = get_prediction_from_watson_ml(6, 6)
print(response)

{
  "fields": ["Heartbeats", "Seconds", "features", "prediction"],
  "values": [[6, 6, [6.0, 6.0], 63.33333333333334]]
}
